In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 100     # number of variables
num_ineq = 100    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 3603.58
Epoch 1, Validation Loss: 7.49
Epoch 2, Validation Loss: 4.89
Epoch 3, Validation Loss: 6.27
Epoch 4, Validation Loss: 6.61
Epoch 5, Validation Loss: 4.04
Epoch 6, Validation Loss: 4.50
Epoch 7, Validation Loss: 4.35
Epoch 8, Validation Loss: 4.31
Epoch 9, Validation Loss: 3.60
Epoch 10, Validation Loss: 3.97
Epoch 11, Validation Loss: 3.52
Epoch 12, Validation Loss: 2.95
Epoch 13, Validation Loss: 3.02
Epoch 14, Validation Loss: 4.51
Epoch 15, Validation Loss: 3.38
Epoch 16, Validation Loss: 2.92
Epoch 17, Validation Loss: 3.61
Epoch 18, Validation Loss: 2.72
Epoch 19, Validation Loss: 2.59
Epoch 20, Validation Loss: 3.34
Epoch 21, Validation Loss: 2.60
Epoch 22, Validation Loss: 2.74
Epoch 23, Validation Loss: 2.97
Epoch 24, Validation Loss: 2.40
Epoch 25, Validation Loss: 3.43
Epoch 26, Validation Loss: 2.46
Epoch 27, Validation Loss: 3.80
Epoch 28, Validation Loss: 2.36
Epoch 29, Validation Loss: 7.78
Epoch 30, Validation Loss: 2.09
Epoch 31, Valid

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_100-100_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 22.36it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     1.772798               0.0      0.003803
std      0.159951               0.0      0.001082
min      1.706022               0.0      0.002005
25%      1.709342               0.0      0.003000
50%      1.718283               0.0      0.003573
75%      1.772511               0.0      0.004508
max      2.726405               0.0      0.008511
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: 3.66
Epoch 2, Validation Loss: 3.35
Epoch 3, Validation Loss: 2.47
Epoch 4, Validation Loss: 2.82
Epoch 5, Validation Loss: 2.97
Epoch 6, Validation Loss: 3.24
Epoch 7, Validation Loss: 2.59
Epoch 8, Validation Loss: 2.45
Epoch 9, Validation Loss: 2.76
Epoch 10, Validation Loss: 1.85
Epoch 11, Validation Loss: 1.93
Epoch 12, Validation Loss: 3.56
Epoch 13, Validation Loss: 2.74
Epoch 14, Validation Loss: 2.23
Epoch 15, Validation Loss: 2.92
Epoch 16, Validation Loss: 2.98
Epoch 17, Validation Loss: 2.41
Epoch 18, Validation Loss: 2.10
Epoch 19, Validation Loss: 2.39
Epoch 20, Validation Loss: 2.03
Epoch 21, Validation Loss: 2.92
Epoch 22, Validation Loss: 2.54
Epoch 23, Validation Loss: 2.22
Epoch 24, Validation Loss: 3.03
Epoch 25, Validation Loss: 2.71
Epoch 26, Validation Loss: 1.88
Epoch 27, Validation Loss: 2.82
Epoch 28, Validation Loss: 2.32
Epoch 29, Validation Loss: 1.94
Epoch 30, Validation Loss: 4.57
Epoch 31, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_100-100_g.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.05it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     1.932418               0.0      0.003507
std      0.059755               0.0      0.001014
min      1.853997               0.0      0.001999
25%      1.905374               0.0      0.003000
50%      1.916547               0.0      0.003221
75%      1.957197               0.0      0.004005
max      2.300400               0.0      0.007221
Number of infeasible solution: 0
